Network anomoly analysis using KDD+IDS dataset 

Importing Necessary Libraries

In [3]:
import numpy as np
import pandas as pd

Loading the dataset and skimming through

In [6]:
lg = pd.read_csv("KDDTrain+_20Percent.csv")

In [8]:
lg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25191 entries, 0 to 25190
Data columns (total 43 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   0         25191 non-null  int64  
 1   tcp       25191 non-null  object 
 2   ftp_data  25191 non-null  object 
 3   SF        25191 non-null  object 
 4   491       25191 non-null  int64  
 5   0.1       25191 non-null  int64  
 6   0.2       25191 non-null  int64  
 7   0.3       25191 non-null  int64  
 8   0.4       25191 non-null  int64  
 9   0.5       25191 non-null  int64  
 10  0.6       25191 non-null  int64  
 11  0.7       25191 non-null  int64  
 12  0.8       25191 non-null  int64  
 13  0.9       25191 non-null  int64  
 14  0.10      25191 non-null  int64  
 15  0.11      25191 non-null  int64  
 16  0.12      25191 non-null  int64  
 17  0.13      25191 non-null  int64  
 18  0.14      25191 non-null  int64  
 19  0.15      25191 non-null  int64  
 20  0.16      25191 non-null  in

In [10]:
lg.head()

,0,tcp,ftp_data,SF,491,0.1,0.2,0.3,0.4,0.5,...,0.17,0.03,0.17.1,0.25,0.26,0.27,0.05,0.28,normal,20
0,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.0,0.00,normal,15
1,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.0,0.00,neptune,19
2,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.0,0.01,normal,21
3,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,normal,21
4,0,tcp,private,REJ,0,0,0,0,0,0,...,0.07,0.07,0.00,0.00,0.00,0.00,1.0,1.00,neptune,21


Adding the variable names

In [13]:
columns = [
    "duration", "protocol_type", "service", "flag", "src_bytes", "dst_bytes", "land", "wrong_fragment", 
    "urgent", "hot", "num_failed_logins", "logged_in", "num_compromised", "root_shell", "su_attempted", 
    "num_root", "num_file_creations", "num_shells", "num_access_files", "num_outbound_cmds", 
    "is_host_login", "is_guest_login", "count", "srv_count", "serror_rate", "srv_serror_rate", 
    "rerror_rate", "srv_rerror_rate", "same_srv_rate", "diff_srv_rate", "srv_diff_host_rate", 
    "dst_host_count", "dst_host_srv_count", "dst_host_same_srv_rate", "dst_host_diff_srv_rate", 
    "dst_host_same_src_port_rate", "dst_host_srv_diff_host_rate", "dst_host_serror_rate", 
    "dst_host_srv_serror_rate", "dst_host_rerror_rate", "dst_host_srv_rerror_rate", 
    "label", "difficulty"
]
lg = pd.read_csv("KDDTrain+_20Percent.csv", names=columns)
lg['binary_label'] = lg['label'].apply(lambda x: 0 if x == 'normal' else 1)
lg.head()


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label,difficulty,binary_label
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20,0
1,0,udp,other,SF,146,0,0,0,0,0,...,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15,0
2,0,tcp,private,S0,0,0,0,0,0,0,...,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19,1
3,0,tcp,http,SF,232,8153,0,0,0,0,...,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21,0
4,0,tcp,http,SF,199,420,0,0,0,0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21,0


Learning the normal behavior and identifying anomolies

In [16]:
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Step 1: Load and filter
columns = [
    "duration", "protocol_type", "service", "flag", "src_bytes", "dst_bytes", "land", "wrong_fragment", 
    "urgent", "hot", "num_failed_logins", "logged_in", "num_compromised", "root_shell", "su_attempted", 
    "num_root", "num_file_creations", "num_shells", "num_access_files", "num_outbound_cmds", 
    "is_host_login", "is_guest_login", "count", "srv_count", "serror_rate", "srv_serror_rate", 
    "rerror_rate", "srv_rerror_rate", "same_srv_rate", "diff_srv_rate", "srv_diff_host_rate", 
    "dst_host_count", "dst_host_srv_count", "dst_host_same_srv_rate", "dst_host_diff_srv_rate", 
    "dst_host_same_src_port_rate", "dst_host_srv_diff_host_rate", "dst_host_serror_rate", 
    "dst_host_srv_serror_rate", "dst_host_rerror_rate", "dst_host_srv_rerror_rate", 
    "label", "difficulty"
]

lg = pd.read_csv("KDDTrain+_20Percent.csv", names=columns)
lg['binary_label'] = lg['label'].apply(lambda x: 0 if x == 'normal' else 1)

# Step 2: One-hot encode categorical features
lg_encoded = pd.get_dummies(lg.drop(columns=['label', 'difficulty', 'binary_label']))

# Step 3: Separate normal-only data for training
X_train = lg_encoded[lg['binary_label'] == 0]

# Step 4: Scale
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_all_scaled = scaler.transform(lg_encoded)

# Step 5: Train Isolation Forest
iso_forest = IsolationForest(contamination=0.05, random_state=42)
iso_forest.fit(X_train_scaled)

# Step 6: Predict on all data
preds = iso_forest.predict(X_all_scaled)  # -1 = anomaly, 1 = normal
lg['anomaly_pred'] = (preds == -1).astype(int)  # 1 = anomaly, 0 = normal


Evaluating the model

In [19]:
from sklearn.metrics import confusion_matrix, classification_report

# Actual and predicted labels
y_true = lg['binary_label']       # Ground truth: 0 = normal, 1 = attack
y_pred = lg['anomaly_pred']       # Predicted: 0 = normal, 1 = anomaly

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
print("📊 Confusion Matrix:\n", cm)

# Classification Report
print("\n📋 Classification Report:")
print(classification_report(y_true, y_pred, digits=4))


📊 Confusion Matrix:
 [[12776   673]
 [ 1222 10521]]

📋 Classification Report:
              precision    recall  f1-score   support

           0     0.9127    0.9500    0.9310     13449
           1     0.9399    0.8959    0.9174     11743

    accuracy                         0.9248     25192
   macro avg     0.9263    0.9229    0.9242     25192
weighted avg     0.9254    0.9248    0.9246     25192



Modeling with parameter tune up to reduce False Negative 

In [43]:
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report, confusion_matrix

# IsolationForest with tuned hyperparameters
#iso = IsolationForest(
#    contamination=0.045,     # Tuned to balance F1 scores
#    max_samples=0.85,        # Slightly under full set to reduce overfitting
#    n_estimators=100,
#    random_state=42
#)

iso = IsolationForest(
    contamination=0.084,      # More aggressive
    max_samples=0.8,         # Use full dataset
    n_estimators=300,        # Stronger ensemble
    max_features=0.81,        # More randomness
    bootstrap=True,          # Optional
    random_state=42
)

# Fit on normal-only data
iso.fit(X_train_scaled)

# Predict on full dataset
lg['anomaly_pred'] = (iso.predict(X_all_scaled) == -1).astype(int)

# Evaluate anomaly detection performance
print("📊 Confusion Matrix")
print(confusion_matrix(lg['binary_label'], lg['anomaly_pred']))
print("\n📋 Classification Report")
print(classification_report(lg['binary_label'], lg['anomaly_pred'], digits=4))


📊 Confusion Matrix
[[12319  1130]
 [  308 11435]]

📋 Classification Report
              precision    recall  f1-score   support

           0     0.9756    0.9160    0.9449     13449
           1     0.9101    0.9738    0.9408     11743

    accuracy                         0.9429     25192
   macro avg     0.9428    0.9449    0.9428     25192
weighted avg     0.9451    0.9429    0.9430     25192



Prediction of the type of anamoly

In [46]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Step 1: Use all flagged records (anomaly_pred == 1)
flagged = lg[lg['anomaly_pred'] == 1].copy()

# Step 2: Drop ultra-rare labels (count ≤ 1) to avoid stratify issues
label_counts = flagged['label'].value_counts()
valid_labels = label_counts[label_counts > 1].index
flagged = flagged[flagged['label'].isin(valid_labels)]

# Step 3: Define features and target
X = pd.get_dummies(flagged.drop(columns=['label', 'difficulty', 'binary_label', 'anomaly_pred']))
y = flagged['label']

# Step 4: Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42
)

# Step 5: Train Random Forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Step 6: Predict and evaluate
y_pred = clf.predict(X_test)

# Step 7: Show results
print("📊 Confusion Matrix")
print(confusion_matrix(y_test, y_pred))
print("\n📋 Classification Report")
print(classification_report(y_test, y_pred, digits=4))


📊 Confusion Matrix
[[   6    0    0    0    0    0    0    0    0    0    0    0    0    0
     0]
 [   0    1    0    0    0    0    0    0    0    0    0    0    0    0
     0]
 [   0    0    2    0    0    0    0    0    0    0    0    0    0    0
     0]
 [   0    0    0    1    0    0    0    0    0    0    0    0    0    0
     0]
 [   0    0    0    0  138    0    1    3    0    0    0    0    0    0
     0]
 [   0    0    0    0    0 1656    0    1    0    0    0    0    0    0
     0]
 [   0    0    0    0    0    0   59    0    0    0    0    0    0    0
     0]
 [   0    0    0    0    0    0    0  226    0    0    0    0    0    0
     0]
 [   0    0    0    0    0    0    0    0    7    0    0    0    0    0
     0]
 [   0    0    0    0    0    1    0    2    0  115    0    0    0    0
     0]
 [   0    0    0    0    0    0    0    3    0    0  134    0    0    0
     0]
 [   0    0    0    0    0    0    0    0    0    0    0  106    0    0
     0]
 [   0    0    0    0

In [48]:
# Predict Stage 2 labels (attack types)
flagged_all = lg[lg['anomaly_pred'] == 1].copy()
X_flagged_all = pd.get_dummies(flagged_all.drop(columns=['label', 'difficulty', 'binary_label', 'anomaly_pred']))
X_flagged_all = X_flagged_all.reindex(columns=X_train.columns, fill_value=0)

predicted_labels = clf.predict(X_flagged_all)

# Add attack_type column to full DataFrame
lg['attack_type'] = 'normal'
lg.loc[flagged_all.index, 'attack_type'] = predicted_labels


# Stage 3: Final prediction logic (Normal vs Abnormal)

In [50]:
from sklearn.metrics import classification_report, confusion_matrix

# Final prediction: Normal if either anomaly_pred == 0 or attack_type == 'normal'
lg['final_pred'] = ((lg['anomaly_pred'] == 1) & (lg['attack_type'] != 'normal')).astype(int)

# Evaluate against ground truth binary_label
print("📊 FINAL Confusion Matrix (Normal vs Abnormal)")
print(confusion_matrix(lg['binary_label'], lg['final_pred']))
print("\n📋 FINAL Classification Report")
print(classification_report(lg['binary_label'], lg['final_pred'], digits=4))


📊 FINAL Confusion Matrix (Normal vs Abnormal)
[[13449     0]
 [  322 11421]]

📋 FINAL Classification Report
              precision    recall  f1-score   support

           0     0.9766    1.0000    0.9882     13449
           1     1.0000    0.9726    0.9861     11743

    accuracy                         0.9872     25192
   macro avg     0.9883    0.9863    0.9871     25192
weighted avg     0.9875    0.9872    0.9872     25192



End of Notebook